In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

In [2]:
class ResizeModel(nn.Module):
    def __init__(self):
        super(ResizeModel, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=270, out_channels=512, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels=512, out_channels=1024, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(in_channels=1024, out_channels=270, kernel_size=3, stride=3)
        self.conv4 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, padding=1)

    def forward(self, x):
        batch_size, time, _, _, _ = x.shape

        # [batch_size, time, transmitter, receiver, subcarrier] -> [batch_size, time, transmitter * receiver * subcarrier]
        x = x.view(batch_size, time, -1)
        # [batch_size, time, transmitter * receiver * subcarrier] -> [batch_size, transmitter * receiver * subcarrier, time]
        x = x.permute(0, 2, 1)

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        # [batch_size, transmitter * receiver * subcarrier, time] -> [batch_size, 1, transmitter * receiver * subcarrier, time]
        x = x.unsqueeze(1)

        # [batch_size, 1, transmitter * receiver * subcarrier, time] -> [batch_size, 3, transmitter * receiver * subcarrier, time]
        x = F.relu(self.conv4(x))

        x = F.interpolate(x, size=(224, 224), mode='bicubic', align_corners=False)

        return x

In [3]:
class VisionTransformer(torch.nn.Module):
    def __init__(self, backbone=None, num_users=6, num_classes=10):
        super(VisionTransformer, self).__init__()

        self.num_users = num_users
        self.num_classes = num_classes

        self.resize_model = ResizeModel()

        self.vit = backbone or models.vit_b_16(weights=None)
        self.vit.heads = torch.nn.Identity()
        self.embed_dim = self.vit.conv_proj.out_channels

        self.head1 = nn.Linear(self.embed_dim, self.num_users * 2)
        self.head2 = nn.Linear(self.embed_dim, self.num_users * self.num_classes)

    def forward(self, x):
        x = self.resize_model(x)
        batch_size, _, _, _ = x.shape

        x = self.vit(x)

        y1, y2 = self.head1(x), self.head2(x)
        y1, y2 = y1.view(batch_size, self.num_users, 2), y2.view(batch_size, self.num_users, self.num_classes)

        return y1, y2

In [4]:
batch_size, time, transmitter, receiver, subcarrier = 4, 3000, 3, 3, 30
x = torch.randn(batch_size, time, transmitter, receiver, subcarrier)

vit = VisionTransformer()

y1, y2 = vit(x)

print("X shape:", x.shape)
print("Y1 shape:", y1.shape)
print("Y1 shape:", y2.shape)

X shape: torch.Size([4, 3000, 3, 3, 30])
Y1 shape: torch.Size([4, 6, 2])
Y1 shape: torch.Size([4, 6, 10])
